# Exploration pour la prédiction des jours Tempo

### Rappel des règles données par RTE pour le placement des jours Tempo: 
- Une année « Tempo » s’étend du 1er septembre d’une année au 31 août de l’année suivante
- Une année Tempo comprend 22 jours rouges et 43 jours blancs
- les jours rouges sont tirés entre le 1er novembre et le 31 mars,
- les jours rouges ne peuvent être tirés le weekend,
- un maximum de 5 jours rouges consécutifs peut être tiré,
- les jours blancs peuvent être tirés toute l’année, sauf le dimanche. 

Source : [Méthode de choix des jours Tempo - Fiche explicative de l’algorithme utilisé par RTE](https://www.services-rte.com/files/live/sites/services-rte/files/pdf/20160106_Methode_de_choix_des_jours_Tempo.pdf)

### Hypothèse / affirmation :  
La production d'éolien et de solaire varie mais est la première brique du mix électrique français. S'ajoute à cette première brique la production du nucléaire, plus grosse base nationale, peu pilotable sur les fluctuations rapides du réseau. Puis viennent charbon et gaz naturel, ce dernier étant le plus pratique pour gérer les fluctuations. 

### Notes :
- Ajouter trois colonnes 'stock_j_blanc', 'stock_j_bleu', 'stock_j_rouge' qui comptent le nombre de jours en stock à J0. Il faut donc prévoir une remise à zéro chaque année.  
=> On pourrait penser que deux colonnes sont suffisantes, mais je pense que le stock de jours bleus peut aussi permettre au modèle de se placer sur le nombre de jours restants sur "l'année Tempo"  
=> Reset les jours chaque année le 1er septembre  <span style="color:green">**Fait**</span>  
- Comment gérer les données manquantes ? On a les données de vent & flux solaire seulement depuis Février 2022.  
=> On peut filtrer sur ces dates là à priori car RTE calcul sur un an glissant à chaque fois <span style="color:green">**Fait**</span>  
- Comment faire comprendre au modèle qu'il ne peut pas prédire un jour rouge dans certain cas ?
=> Ajouter un booléen pour signifier si le jour peut être rouge ou non ?
=> Est-il possible de donner le choix uniquement entre jours blancs et jour bleus les jours quand on est en dehors des conditions pour les jours rouges ?
=> Ou alors encore plus simple : si un jour rouge est prédit, mais les conditions ne sont pas remplies = jour blanc (si stock restant), RTE bosse de cette manière. <span style="color:red">**A faire**</span>  
- Quelles colonnes garder pour le modèle ?
=> Les stocks de jour, la production à J-1, la consommation à J+1, la production ENR (ou flux solaire / vent) à J+1 <span style="color:red">**A faire**</span>  


In [38]:
import pandas as pd
pd.set_option('display.max_rows', 400)

import os
import plotly.express as px
from calendar import isleap


## Read data

In [39]:
in_relative_path = "../../../data/gold/"
in_absolute_path = os.path.abspath(os.path.join(os.getcwd(), in_relative_path))

data = pd.read_csv(f"{in_absolute_path}/rte_daily_2014_2024.csv")
data["Date"] = pd.to_datetime(data["Date"])


# Feature Engineering
### Ajout des colonnes de stocks restants des jours Tempo

In [40]:
stock_jours_annuels = {
    "rouge": 22,
    "blanc": 43,
    "bleu": 300
}
stock_jours_annuels_b = {
    "rouge": 22,
    "blanc": 43,
    "bleu": 301
}

def get_base_stock(year:int) -> dict:
    """
    Vérifie si l'année est bissextiles ou non
    Renvoie le stock initial de jour tempo.

    :param year: _description_
    :return: _description_
    """
    if isleap(year):
        return stock_jours_annuels_b.copy()
    else:
        return stock_jours_annuels.copy()
    
def update_stock(previous_tempo:str, stock_actuel:dict) -> dict:
    if previous_tempo:
        if previous_tempo == "ROUGE":
            stock_actuel["rouge"] -= 1
        elif previous_tempo == "BLANC":
            stock_actuel["blanc"] -= 1
        elif previous_tempo == "BLEU":
            stock_actuel["bleu"] -= 1
        return stock_actuel
    else:
        return None
    
def check_stock_empty(stock_actuel:dict, year:int):
    """
    Cette fonction permet de vérifier si les stocks sont bien calculés :
    Tout les stocks doivent à 0, si ce n'est pas le cas c'est que RTE
    n'a pas utilisé tout les jours en stock, l'écart devrait alors se 
    répercuter négativement sur les jours bleus (la somme du dictionnaire doit être de 0 forcément)

    :param stock_actuel: _description_
    :param year: _description_
    """
    if stock_actuel:
        if stock_actuel["rouge"] == stock_actuel["bleu"] == stock_actuel["blanc"] == 0:
            pass
        else:
            print(f"Les stocks de jours n'était pas à 0 le 1er septembre {year} ! :(")
            print(f"Voici l'état du stock : {stock_actuel}")
            if stock_actuel["rouge"] + stock_actuel["bleu"] + stock_actuel["blanc"] != 0:
                raise ValueError("La somme des stocks ne fait pas 0 à la fin de l'année.")


def get_cumulative_stock(data):
    """
    On calcule les stocks restants pour chaque jour tempo à une date X.

    Les valeurs représentent la situation à J-1 car la prédiction
    pour une date D se fait avec le stocks restants à J-1.

    Ex : 
    Une row avec 
        - stock_j_rouge = 0
        - stock_j_blanc = 1
        - stock_j_bleu = 14
        - type_tempo = "BLANC"
    => Au moment de prédire le jour "BLANC", je n'avais plus de jour rouge
    mais il me restait un dernier jour blanc en stock

    :param data: _description_
    :return: _description_
    """
    stock_rouge, stock_bleu, stock_blanc = [], [], []
    previous_tempo = stock_actuel = None
    for i, row in data.iterrows():
        # Remise à zéro les 1er Septembre
        # sinon on update le stock en fonction du type tempo de la veille 
        if row["Date"].month == 9 and row["Date"].day == 1:
            update_stock(previous_tempo, stock_actuel)
            check_stock_empty(stock_actuel, row["Date"].year)
            stock_actuel = get_base_stock(row["Date"].year + 1)
        else:
            update_stock(previous_tempo, stock_actuel)
        
        stock_rouge.append(stock_actuel["rouge"])
        stock_blanc.append(stock_actuel["blanc"])
        stock_bleu.append(stock_actuel["bleu"])

        # On enregistre le type tempo
        previous_tempo = row["type_tempo"]
    
    data["stock_j_rouge"] = stock_rouge
    data["stock_j_blanc"] = stock_blanc
    data["stock_j_bleu"] = stock_bleu
    return data

data = get_cumulative_stock(data)

Les stocks de jours n'était pas à 0 le 1er septembre 2020 ! :(
Voici l'état du stock : {'rouge': 4, 'blanc': -4, 'bleu': 0}
Les stocks de jours n'était pas à 0 le 1er septembre 2022 ! :(
Voici l'état du stock : {'rouge': 0, 'blanc': 1, 'bleu': -1}


# Filtre sur les dernières données
### On garde uniquement les données avec des valeurs de vent & flux solaire

In [14]:
data = data[data["Date"] >= "2022-02-01"].reset_index(drop=True)

In [15]:
data

,Date,Nature,type_tempo,Consommation,Prévision_J-1,Prévision_J,Fioul,Charbon,Gaz,Nucléaire,...,Pompage,Bioénergies,Ech_physiques,Taux_de_Co2,Ech_comm,sun,wind,stock_j_rouge,stock_j_bleu,stock_j_blanc
0,2022-02-01,Données consolidées,BLEU,860492.500,842175.00,847500.0,2064.500,5281.50,82777.750,571452.750,...,-11918.250,13914.000,14484.250,554.500,19324.750,3.901272e+06,4.906875,0,189,22
1,2022-02-02,Données consolidées,BLANC,805681.500,807350.00,795900.0,1679.500,6394.25,89255.250,573210.000,...,-12251.750,14048.750,-12253.750,620.250,-8235.750,4.494408e+06,3.661905,0,189,21
2,2022-02-03,Données consolidées,BLANC,809729.000,803075.00,802310.0,2067.750,6734.00,91275.000,578999.250,...,-8398.000,14391.000,10752.250,653.500,15055.000,4.781211e+06,2.776255,0,189,20
3,2022-02-04,Données consolidées,BLEU,796406.000,788950.00,779700.0,1727.500,7230.75,81583.750,575861.750,...,-11451.250,14844.000,-44027.500,576.500,-39605.750,4.245190e+06,3.987258,0,188,20
4,2022-02-05,Données consolidées,BLEU,743808.500,725400.00,734900.0,1756.750,268.00,61683.250,538712.500,...,-8911.000,15130.500,-17546.000,422.000,-14231.000,7.963900e+06,3.375999,0,187,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,2024-04-05,Données temps réel,BLEU,556538.000,561437.50,550637.5,1499.000,0.00,7277.250,407607.375,...,-15021.250,9543.000,-101724.500,160.375,-49060.250,1.423090e+07,4.879554,0,139,8
795,2024-04-06,Données temps réel,BLEU,485519.875,478718.75,486987.5,1498.000,0.00,7017.750,324737.250,...,-15857.375,9473.625,-77274.750,183.875,-38609.625,1.757204e+07,5.793304,0,138,8
796,2024-04-07,Données temps réel,BLEU,464582.375,465193.75,465375.0,1633.375,0.00,7946.625,308369.250,...,-16658.375,9422.625,-26822.250,223.250,-13769.250,1.166932e+07,3.579645,0,137,8
797,2024-04-08,Données temps réel,BLEU,290497.750,544651.00,544925.0,821.500,0.00,4534.875,243594.250,...,-5179.250,5255.625,-56747.375,97.250,-51114.125,1.391853e+07,4.507709,0,136,8
